# Directory of Vizualisations


This section is a quick overview of various types of plots and charts typically used in data visualisation. We will provide guidance of which types of plots are best different types of data and hints of what to be careful of when using a given chart. 


We emphasise that figures are a graphical layer on top of your data. **The nature of the figure should be dictated by the nature of the data**.

Though different types of data lend itself to different styles of chart, the figure components discussed in _Rules of the Data Vizualisation Game_ applies to all figures. All figures should be appropriately scaled, coloured, and labelled.


[hackmd notes](https://hackmd.io/iO0-zr1xQWqC097-neJxSA?view)

## Amounts

Here we are interested in the magnitude of some set of number: We have a set of categories and a quantitative value for each category. We show four broad types of data.